# TODO


1.   Натренировать Relation Extraction
2.   Добавить early_stopping
3.   Save-n models
5.   Совместить с NEREL

# DONE

In [ ]:
ls

drive/  sample_data/


In [ ]:
!git clone https://github.com/denis-gordeev/rured2

Cloning into 'rured2'...
remote: Enumerating objects: 4486, done.
remote: Counting objects: 100% (4486/4486), done.
remote: Compressing objects: 100% (3911/3911), done.
remote: Total 4486 (delta 1065), reused 3981 (delta 570), pack-reused 0
Receiving objects: 100% (4486/4486), 4.10 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (1065/1065), done.


In [ ]:
!nvidia-smi

Thu Nov 10 15:14:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install razdel==0.5.0 >> ./pip.log
!pip install seqeval==1.2.2 >> ./pip.log
!pip install datasets==2.2.2 >> ./pip.log
!pip install fire==0.4.0 >> ./pip.log
!pip install boto3==1.24.6 >> ./pip.log
!pip install transformers >> ./pip.log

In [ ]:
!ls rured2/data/annotation_files | head
!echo "Total $(ls rured2/data/annotation_files | wc -l) files"

101144.ann
101144.txt
102083.ann
102083.txt
10223.ann
10223.txt
104927.ann
104927.txt
11035.ann
11035.txt
Total 1054 files


In [ ]:
from glob import glob
import shutil

In [ ]:
# I have mixed automatically and manually labelled files here
# Currently only manually annotated files contain relations
ann_files = glob("rured2/data/rbc/*/*.ann")
for ann_file in ann_files:
    with open(ann_file) as f:
        annotation = f.readlines()
    if not any(l.startswith("R") for l in annotation):
        continue
    txt_file = ann_file.replace(".ann", ".txt")
    shutil.copy(ann_file, "rured2/data/annotation_files/")
    shutil.copy(txt_file, "rured2/data/annotation_files/")

In [ ]:
!ls rured2/data/annotation_files | head
!echo "Total $(ls rured2/data/annotation_files | wc -l) files"

101144.ann
101144.txt
102083.ann
102083.txt
10223.ann
10223.txt
104927.ann
104927.txt
11035.ann
11035.txt
Total 1328 files


In [ ]:
# !git clone --recurse-submodules https://github.com/denis-gordeev/rured/

In [ ]:
# ls ./rured/SpanBERT/

In [ ]:
# !python ./rured/convert_brat_to_tacred.py "rured2/data/annotation_files/*.ann"

In [ ]:
# ls -lah all_data.json

In [ ]:
from glob import glob
from typing import Dict, List, Optional

import pandas as pd
from sklearn.model_selection import train_test_split
import razdel
import torch
from torch import nn
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, top_k_accuracy_score

from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForTokenClassification, 
                          TrainingArguments, Trainer)
from transformers import EarlyStoppingCallback

In [ ]:
# model_ckpt = "cointegrated/rubert-tiny2"
# model_ckpt = "yandex/RuLeanALBERT"
# model_ckpt = "xlm-roberta-base"
model_ckpt = "bert-base-multilingual-cased"

In [ ]:
def check_pair_in_range(start: int, end: int, check_range: range) -> bool:
    if range(max(start, check_range.start), min(end, check_range.stop)):
        return True
    else:
        return False

def get_annotation_from_file(
    annotation_filename: str, with_relations_only: bool = False
) -> List[dict]:
    with open(annotation_filename) as f:
        annotation_str: str = f.read()
    annotation_values: List[dict] = []
    if with_relations_only and "Arg1:" not in annotation_str:
        return []
    annotation: List = annotation_str.split("\n")
    annotation = [a.strip() for a in annotation]
    annotation = [a for a in annotation if a]
    annotation = [a.split("\t") for a in annotation]
    annotation_entries: dict = dict()
    ners = [a for a in annotation if a[0][0] == "T"]
    relations = [a for a in annotation if a[0][0] == "R"]
    annotation = ners + relations
    for a in annotation:
        # error annotations
        # TODO fix
        if len(a) < 2:
            continue
        annotation_entry: dict = dict()
        if a[0].startswith("T"):
            annotation_type = "NER"
        elif a[0].startswith("R"):
            annotation_type = "Relation"
        else:
            annotation_type = "Unknown"
        annotation_entry["type"] = annotation_type
        annotation_entry["code"] = annotation_code = a[0]

        annotation_body = a[1]
        annotation_body = annotation_body.replace("Arg1:", "").replace(
            "Arg2:", ""
        )
        annotation_body = annotation_body.split()
        annotation_entry["ann_type"] = annotation_body[0]
        if annotation_type == "NER":
            if len(a) <= 2:
                continue
            annotation_entry["start"] = int(annotation_body[1])
            annotation_entry["end"] = int(annotation_body[2])
            annotation_entry["text"] = a[2]
        else:
            for item_id, _ in enumerate(("subj", "obj")):
                item_id += 1
                for position in ("start", "end"):
                    annotation_entry[
                        f"{item_id}_{position}"
                    ] = annotation_entries[annotation_body[item_id]][position]
            annotation_entry["subj_ref"] = annotation_body[1]
            annotation_entry["obj_ref"] = annotation_body[2]
        annotation_entries[annotation_code] = annotation_entry
    annotation_values = list(annotation_entries.values())
    return annotation_values

def tokenize_and_align_labels(example, text_column_name="text", label_column_name="ner", padding=True, label_all_tokens=True, max_seq_length=512):
    # https://github.com/huggingface/transformers/blob/main/examples/pytorch/token-classification/run_ner.py#L421
    texts = example[text_column_name]
    if type(sents_df["text"]) == pd.core.series.Series:
        texts = list(texts.values)
    tokenized_inputs = tokenizer(
        texts,
        padding=padding,
        truncation=True,
        max_length=max_seq_length,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(example[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            new_label = [0] * len(id_to_label)
            if word_idx is None:
                label_ids.append(new_label)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                for ner in label[word_idx]:
                    new_label[label_to_id[ner]] = 1
                label_ids.append(new_label)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                if label_all_tokens:
                    label_ids.append(b_to_i_label[label_to_id[label[word_idx]]])
                else:
                    label_ids.append(new_label)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
# def create_ner_dataset(glob_path="rured2/data/annotation_files/*.ann"):
    
glob_path="rured2/data/annotation_files/*.ann"
filenames = glob(glob_path)
text_data = dict()
for annotation_filename in filenames:
    text_filename = annotation_filename.replace(".ann", ".txt")
    annotation_values = get_annotation_from_file(annotation_filename)
    with open(text_filename) as f:
        text = f.read()
    sents = list(razdel.sentenize(text))
    prev_sent = None
    text_data[annotation_filename] = []
    attach_flag = False
    for sent in sents:
        if attach_flag and prev_sent:
            sent.text = prev_sent.text + " " + sent.text
            sent.start = prev_sent.start
            attach_flag = False
            prev_sent = None
        sent_tokens = list(razdel.tokenize(sent.text))
        sent_ners = [["O"] for t in sent_tokens]
        # sent_brat_codes = [["O"] for t in sent_tokens]
        sent_data: Dict = dict()
        sent_data["token"] = [t.text for t in sent_tokens]
        sent_data["text"] = sent.text
        # sent_data["relation"] = False
        # sent_data["relations"] = []
        for _, value in enumerate(annotation_values):
            annotation_code = value["code"]
            if value["type"] == "NER":
                start = value["start"]
                end = value["end"]
                ner_text = value["text"]
                ner_type = value["ann_type"]
                overlaps = check_pair_in_range(
                    start, end, range(sent.start, sent.stop)
                )
                if overlaps:
                    for s_t_i, s_t in enumerate(sent_tokens):
                        token_overlaps = check_pair_in_range(
                            start,
                            end,
                            range(
                                s_t.start + sent.start,
                                s_t.stop + sent.start,
                            ),
                        )
                        if token_overlaps:
                            if sent_ners[s_t_i] == ['O']:
                                sent_ners[s_t_i] = []
                            if ner_text.startswith(s_t.text):
                                sent_ners[s_t_i].append("B-" + ner_type)
                            else:
                                sent_ners[s_t_i].append("I-" + ner_type)

        sent_data["stanford_ner"] = sent_ners
        # sent_data["brat_codes"] = sent_brat_codes
        sent_data["start"] = sent.start
        sent_data["stop"] = sent.stop
        # all_data.append(sent_data)
        text_data[annotation_filename].append(sent_data)

In [ ]:
sents = []
for filename, text in text_data.items():
    for sent in text:
        sent["filename"] = filename
        sents.append(sent)

In [ ]:
label_to_id = {"O": 0}
id_to_label = {0: "O"}
new_label = 1

for sent in sents:
    for token_ners in sent["stanford_ner"]:
        for ner in token_ners:
            if ner not in label_to_id:
                label_to_id[ner] = new_label
                id_to_label[new_label] = ner
                new_label += 1
print(id_to_label)

{0: 'O', 1: 'B-COUNTRY', 2: 'B-LAW', 3: 'B-ORGANIZATION', 4: 'I-LAW', 5: 'I-ORGANIZATION', 6: 'B-DATE', 7: 'I-DATE', 8: 'B-NEWS_SOURCE', 9: 'I-NEWS_SOURCE', 10: 'B-GROUP', 11: 'I-GROUP', 12: 'B-PRODUCT', 13: 'I-PRODUCT', 14: 'B-PERCENT', 15: 'I-PERCENT', 16: 'B-PERSON', 17: 'I-PERSON', 18: 'B-PROFESSION', 19: 'I-PROFESSION', 20: 'B-QUANTITY', 21: 'I-QUANTITY', 22: 'B-ORDINAL', 23: 'B-MONEY', 24: 'I-MONEY', 25: 'B-CURRENCY', 26: 'B-REGION', 27: 'I-REGION', 28: 'B-CARDINAL', 29: 'I-CARDINAL', 30: 'B-TIME', 31: 'B-CITY', 32: 'I-CITY', 33: 'B-COMMODITY', 34: 'B-ECONOMIC_SECTOR', 35: 'I-ECONOMIC_SECTOR', 36: 'B-NORP', 37: 'B-GPE', 38: 'B-EVENT', 39: 'I-GPE', 40: 'B-FAC', 41: 'I-FAC', 42: 'B-LOCATION', 43: 'I-LOCATION', 44: 'I-COUNTRY', 45: 'B-VEHICLE', 46: 'I-EVENT', 47: 'I-TIME', 48: 'B-AGE', 49: 'B-DISEASE', 50: 'I-DISEASE', 51: 'B-CRIME', 52: 'I-CRIME', 53: 'B-FAMILY', 54: 'B-BOROUGH', 55: 'I-COMMODITY', 56: 'B-WORK_OF_ART', 57: 'I-WORK_OF_ART', 58: 'B-INVESTMENT_PROGRAM', 59: 'I-INVESTM

In [ ]:
import json
with open("id_to_label.json", "w") as f:
    json.dump(id_to_label, f, ensure_ascii=False)

In [ ]:
sents_df = pd.DataFrame(sents)
sents_df.head(5)

,token,text,stanford_ner,start,stop,filename
0,"[Постановление, об, отмене, госрегулирования, ...",Постановление об отмене госрегулирования цен н...,"[[O], [O], [O], [O], [O], [O], [O], [O], [B-CO...",0,257,rured2/data/annotation_files/69298.ann
1,"[С, этого, дня, украинские, производители, про...",С этого дня украинские производители продуктов...,"[[O], [O], [O], [B-COUNTRY], [B-GROUP], [I-GRO...",258,411,rured2/data/annotation_files/69298.ann
2,"[Согласно, документу, ,, производители, могут,...","Согласно документу, производители могут устана...","[[O], [O], [O], [B-GROUP], [O], [O], [O], [O],...",412,554,rured2/data/annotation_files/69298.ann
3,"[Ранее, такие, надбавки, к, ценам, на, хлеб, ,...","Ранее такие надбавки к ценам на хлеб, муку, кр...","[[O], [O], [O], [O], [O], [O], [B-PRODUCT], [O...",555,857,rured2/data/annotation_files/69298.ann
4,"[По, мнению, Владимира, Сиденко, ,, эксперта, ...","По мнению Владимира Сиденко, эксперта экономич...","[[B-NEWS_SOURCE], [I-NEWS_SOURCE], [B-PERSON, ...",858,1039,rured2/data/annotation_files/69298.ann


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
sents_train, sents_test = train_test_split(sents_df, test_size=0.1, random_state=42)

In [ ]:
tokenized_train = tokenize_and_align_labels(sents_train, "token", "stanford_ner", label_all_tokens=False)
tokenized_test = tokenize_and_align_labels(sents_test, "token", "stanford_ner", label_all_tokens=False)

In [ ]:
# hf_dataset = Dataset.from_dict(tokenized_inputs)
hf_train = Dataset.from_dict(tokenized_train)
hf_test = Dataset.from_dict(tokenized_test)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions = (predictions > 0.5).astype(int)
    # f1_results = f1_score(labels,
    #                       (predictions > 0.5).astype(int),
    #                       average=None)
    # print("pred0", predictions[0])
    # print("pred0sampl", (predictions[0] > 0.5).astype(int))
    # print("label0", labels[0])

    # print("preds", predictions)
    # print("labels", labels)
    f1_micro = 0
    f1_macro = 0
    for i, l in enumerate(labels):
        f1_micro += f1_score(l,
                             (predictions[i] > 0.5).astype(int),
                             # (predictions > 0.5).astype(int),
                             average="micro",
                             zero_division=0)
        f1_macro += f1_score(l,
                            (predictions[i] > 0.5).astype(int),
                            # (predictions > 0.5).astype(int),
                            average="macro",
                            zero_division=0)
    f1_micro /= predictions.shape[0]
    f1_macro /= predictions.shape[0]
    output = {'f1_macro': f1_macro,
              'f1_micro': f1_micro}
    # for k, v in label_to_id.items():
    #     output[f"f1_{k}"] = f1_results[v]
    return output

class MultiLabelNERTrainer(Trainer):
    def __init__(self, *args, class_weights = None, **kwargs):
        super().__init__(*args, **kwargs)
        if class_weights is not None:
            class_weights = class_weights.to(self.args.device)
            logging.info(f"Using multi-label classification with class weights", class_weights)
        self.loss_fct = torch.nn.BCEWithLogitsLoss(weight=class_weights)

    def compute_loss(self, model, inputs, return_outputs=False):
        """
        How the loss is computed by Trainer. By default, all models return the loss in the first element.
        Subclass and override for custom behavior.
        """
        labels  = inputs.pop("labels")
        outputs = model(**inputs)
        # print(outputs)
        # this accesses predictions for tokens that aren't CLS, PAD, or the 2nd+ subword in a word
        # and simultaneously flattens the logits or labels
        # print("LOGITS", outputs.logits)
        # print("LABELS", labels)
        # print("labels 0", labels[0])
        # print("LOGITS 0", outputs.logits[0])
        # print("labels 0", labels[0].shape)
        # print("LOGITS 0", outputs.logits[0].shape)
        # flat_outputs = outputs.logits[labels > 0] 
        # flat_outputs = outputs.logits.view(8 * 340, -1)[(labels.sum(axis=2) > 2).view(-1)].view(8, -1, 76)
        flat_outputs = outputs.logits[labels.sum(axis=2) > 0]
        flat_labels  = labels[labels.sum(axis=2) > 0]
        # print("flat_outputs", flat_outputs.shape)
        # print("flat_labels", flat_labels.shape)
        try:
            loss = self.loss_fct(flat_outputs, flat_labels.float())
        except AttributeError:  # DataParallel
            loss = self.loss_fct(flat_outputs, flat_labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
# Make sure your data is correctly formatted, e.g., 0 or 1 encoded for each label for each sample.

In [ ]:
ls

drive/  id_to_label.json  pip.log  rured2/  sample_data/


In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_ckpt, num_labels=len(id_to_label))#.to('cuda')
model.config.label2id = label_to_id
model.config.id2label = id_to_label

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
training_args = TrainingArguments(
    output_dir="multilabel_ner",
    save_strategy="epoch",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10000,
    weight_decay=0.01,
    save_total_limit=20,
    metric_for_best_model="f1_macro",
    load_best_model_at_end=True,
    fp16=True
)

trainer = MultiLabelNERTrainer(
    model=model,
    args=training_args,
    train_dataset=hf_train,
    eval_dataset=hf_test,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

for param in model.parameters():
    param.requires_grad = False

dataset_name = "rured2_model"
# gradual unfreezing
for i, param in enumerate(list(model.parameters())[::-1]):
    param.requires_grad = True
    if i % 40 != 0:
        continue
    print(i, len(param.shape), len(list(model.parameters())))
    trainer.evaluate()
    trainer.train()
    results = trainer.evaluate()
    prev_score = results["eval_f1_macro"]
    best_model_checkpoint = None
    if results["eval_f1_macro"] < prev_score and i >= 2:
        print(f"loading previous best model: {best_model_checkpoint}")
        trainer.state.best_model_checkpoint = best_model_checkpoint
        trainer._load_best_model()
    else:
        prev_score = results["eval_f1_macro"]
        best_model_checkpoint = f"{dataset_name}_step{i}"
        print(f"saving new best model: {best_model_checkpoint}")
        trainer.save_model(f"{dataset_name}_step{i}")

trainer.save_model("multilabel_ner_rured2")

Using cuda_amp half precision backend
***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


0 1 199


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7641
  Num Epochs = 10000
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4780000
  Number of trainable parameters = 88


Epoch,Training Loss,Validation Loss,F1 Macro,F1 Micro
1,No log,0.700500,0.001625,0.008282
2,0.704200,0.698171,0.001647,0.008656
3,0.701700,0.695851,0.001655,0.009053
4,0.699400,0.693537,0.001641,0.009426
5,0.696900,0.691230,0.001634,0.009820
6,0.694400,0.688930,0.001649,0.010235


***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-478
Configuration saved in multilabel_ner/checkpoint-478/config.json
Model weights saved in multilabel_ner/checkpoint-478/pytorch_model.bin
tokenizer config file saved in multilabel_ner/checkpoint-478/tokenizer_config.json
Special tokens file saved in multilabel_ner/checkpoint-478/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-956
Configuration saved in multilabel_ner/checkpoint-956/config.json
Model weights saved in multilabel_ner/checkpoint-956/pytorch_model.bin
tokenizer config file saved in multilabel_ner/checkpoint-956/tokenizer_config.json
Special tokens file saved in multilabel_ner/checkpoint-956/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-1434
Configuration 

...

***** Running training *****
  Num examples = 7641
  Num Epochs = 10000
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4780000
  Number of trainable parameters = 18968152


Epoch,Training Loss,Validation Loss,F1 Macro,F1 Micro
1,No log,0.083925,0.002650,0.215651
2,0.199300,0.045187,0.002648,0.210655
3,0.061600,0.031662,0.003094,0.241854
4,0.038200,0.024425,0.006204,0.259277


***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-478
Configuration saved in multilabel_ner/checkpoint-478/config.json
Model weights saved in multilabel_ner/checkpoint-478/pytorch_model.bin
tokenizer config file saved in multilabel_ner/checkpoint-478/tokenizer_config.json
Special tokens file saved in multilabel_ner/checkpoint-478/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-956
Configuration saved in multilabel_ner/checkpoint-956/config.json
Model weights saved in multilabel_ner/checkpoint-956/pytorch_model.bin
tokenizer config file saved in multilabel_ner/checkpoint-956/tokenizer_config.json
Special tokens file saved in multilabel_ner/checkpoint-956/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-1434
Configuration 

Epoch,Training Loss,Validation Loss,F1 Macro,F1 Micro
1,No log,0.083925,0.002650,0.215651
2,0.199300,0.045187,0.002648,0.210655
3,0.061600,0.031662,0.003094,0.241854
4,0.038200,0.024425,0.006204,0.259277
5,0.028400,0.019874,0.010824,0.275127
6,0.022600,0.016794,0.016370,0.288171
7,0.018900,0.014620,0.021533,0.297986
8,0.016300,0.013105,0.024462,0.303272
9,0.014500,0.011878,0.025564,0.304862
10,0.012900,0.010877,0.025896,0.303417


***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-2390
Configuration saved in multilabel_ner/checkpoint-2390/config.json
Model weights saved in multilabel_ner/checkpoint-2390/pytorch_model.bin
tokenizer config file saved in multilabel_ner/checkpoint-2390/tokenizer_config.json
Special tokens file saved in multilabel_ner/checkpoint-2390/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-2868
Configuration saved in multilabel_ner/checkpoint-2868/config.json
Model weights saved in multilabel_ner/checkpoint-2868/pytorch_model.bin
tokenizer config file saved in multilabel_ner/checkpoint-2868/tokenizer_config.json
Special tokens file saved in multilabel_ner/checkpoint-2868/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 849
  Batch size = 16
Saving model checkpoint to multilabel_ner/checkpoint-3346
Conf

In [ ]:
trainer.save_model("bert_rured_training")

In [ ]:
!zip bert_rured_training.zip bert_rured_training/*
!cp bert_rured_training.zip drive/MyDrive/

In [ ]:
cp drive/MyDrive/multilabel_ner_rured2_sept30_2022.zip ./

In [ ]:
!unzip multilabel_ner_rured2_sept30_2022.zip

In [ ]:
import json
with open("multilabel_ner_rured2/id_to_label.json") as f:
    id_to_label = json.load(f)
id_to_label = {int(k): v for k, v in id_to_label.items()}

In [ ]:
model_desc = model_ckpt.replace("/", "_")
model = AutoModelForTokenClassification.from_pretrained(
    f"multilabel_ner_rured2_{model_ckpt}") #.to('cuda')
model.config.id2label = id_to_label

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def predict(text:str):
    tokenized = tokenizer(text)
    input_ids = torch.tensor(
            [tokenized["input_ids"]], dtype=torch.long
        )#.to("cuda")
    token_type_ids = torch.tensor(
            [tokenized["token_type_ids"]], dtype=torch.long
        )#.to("cuda")
    attention_mask = torch.tensor(
            [tokenized["attention_mask"]], dtype=torch.long
        )#.to("cuda")
    preds = model(**{"input_ids": input_ids, "token_type_ids": token_type_ids, "attention_mask": attention_mask})
    logits = preds.logits
    for i, token in enumerate(input_ids[0]):
        if token > 3:
            class_ids = (logits[0][i] > -3).nonzero()
            if class_ids.shape[0] >= 1:
                class_names = [id_to_label[int(cl)] for cl in class_ids]
            else:
                class_names = [id_to_label[int(logits[0][i].argmax())]]
        else:
            class_names = []
        print(
            tokenizer.convert_ids_to_tokens([token]),
            class_names
        )

In [ ]:
texts = [
    "Газпром продает нефть Украине",
    "Согласно Reuters продажа газа Россией упала на 4%.",
    "IPhone 10 - последняя модель от Apple",
    "Рогозин объявил о завершении военной операции в Сирии",
    "Число рабочих мест в Санкт-Петербурге будет увеличено на 1000 в связи с вовзедением завода Уралзаводвагмаш"
]
for text in texts:
    predict(text)
    print(("-" * 80 + "\n")*2)

In [ ]:
predict(
    'Компания «Читинские ключи» получила господдержку и запустила производство «Колы» для сохранения рабочих мест, '
    'сообщили в пресс-службе министерства экономического развития Забайкальского края. По словам замминистра экономического '
    'развития Забайкалья Дениса Рысева, у &laquo;Колы&raquo; особенный вкус. На производстве уверяют, что используется секретный '
    'ингредиент. Как и у той &laquo;Кока-колы&raquo;, известной на весь мир. Благодаря запуску нового производства компании удалось '
    'сохранить 241 сотрудника, сообщает РИА &laquo;Новости&raquo;. Уточняется, что первая партия &laquo;Колы&raquo; – 900 упаковок '
    'была изготовлена 5 октября. В этот же день 600 упаковок купили предприниматели региона для продажи в магазинах. &laquo;Это '
    'ожидаемый продукт в условиях импортозамещения. Скучали по &laquo;Кока-коле&raquo;? Вот же она –&nbsp;только еще лучше, своя! '
    'Гарантийный фонд Забайкалья стал поручителем по кредиту для компании &laquo;Читинские ключи&raquo;. Сумма поручительства 25 '
    'миллионов рублей, сумма кредита –&nbsp;почти 50 миллионов рублей&raquo;, – добавил Рысев. Ранее сообщалось, что в Московской '
    'области планируется запустить новое производство компрессоров и фильтров для жидкостей, этим займется компания &laquo;Полинет&raquo;.',)

In [ ]:
predict('В Новосибирске задержали безработного жителя Томска, который продавал наркотики через интернет-магазин')

In [ ]:
predict('В России стали заменять одежду брендов H&amp;M и Zara дорогими аналогами из Турции. Об этом сообщает издание NEWS.ru. По словам анонимного источника портала, среди подобных альтернатив оказались такие турецкие марки, как Perspective, Ketroy и Mexx, магазины которого закрылись в стране в 2017 году из-за нерентабельности.')

In [ ]:
!cp id_to_label.json multilabel_ner_rured2/
!zip multilabel_ner_rured2_sept30_2022.zip multilabel_ner_rured2/*
!cp multilabel_ner_rured2_sept30_2022.zip drive/MyDrive 